In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Trend実験

In [4]:
# mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=False, coord=False, co_est=False, fix_pop=False, add_location=False, outlier=False)
# df_all = preprocess.add_lag_features(df_all)

/tmp/working/code-analysis/../modules/utils.py:293: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


In [5]:
df_census

,adult_2017,cfips,adult_2018,adult_2019,adult_2020,adult_2021
0,41527,1001,41831,42175,42496.0,44438.0
1,158361,1003,162430,166595,171296.0,178105.0
2,20687,1005,20346,20054,19804.0,19995.0
3,17852,1007,17868,17862,17790.0,17800.0
4,44102,1009,44177,44292,44383.0,45201.0
...,...,...,...,...,...,...
3215,42989,72145,42353,41843,41383.0,44604.0
3216,7051,72147,6985,6946,6884.0,6760.0
3217,18113,72149,17809,17547,17344.0,17811.0
3218,27598,72151,27227,27029,26803.0,25533.0


In [170]:
low_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0,
        'lower_bound': 60,
        'upper_bound': 140,
        'use_regularize': True,
        'v_regularize': [0.03, 0.02],
        'v_clip':[0.999, 1.004]
    }
}
params = low_trend_params[1]
df_trend, trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(trend_dict))
trend_dict

13


{13025: 1.004,
 13193: 0.999,
 18025: 0.999,
 21147: 1.004,
 31093: 1.0033846255390906,
 40005: 1.004,
 40069: 1.00303696951699,
 46091: 1.0016915440454517,
 48305: 1.004,
 48385: 1.0026579234749724,
 5067: 1.004,
 54023: 0.999,
 8033: 1.004}

In [7]:
high_trend_params = {
    1: {
        'n':3,
        'thre':3,
        'thre_r':0.004,
        'lower_bound': 20000,
        'upper_bound': 99999999,
        'use_regularize': True,
        'v_regularize': [0.002, 0],
        'v_clip':[None, None]
    }
}
params = high_trend_params[1]
df_trend, high_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(high_trend_dict))
high_trend_dict

20


{12031: 1.0086328445657107,
 12101: 1.006481142747778,
 17197: 1.0080206078421285,
 20091: 1.0060327304334955,
 25025: 1.0061688635360524,
 36085: 1.0173209335038416,
 37119: 1.006098418566488,
 40109: 1.0129319463023307,
 40143: 1.008331800893501,
 42003: 1.0233533145463996,
 47065: 1.008465009610966,
 47187: 1.0075341777021485,
 48113: 1.0045559762608431,
 48121: 1.0118033817358585,
 48157: 1.013289253684673,
 48491: 1.0074687019144728,
 55133: 1.0069273342745635,
 6001: 1.0116187765330433,
 6073: 1.0100269518719414,
 6079: 1.0050638864187709}

In [163]:
middle_trend_params = {
    1: {
        'n':7,
        'thre':7,
        'thre_r':0.001,
        'lower_bound': 500,
        'upper_bound': 20000,
        'use_regularize': True,
        'v_regularize': [0.001, 0],
        'v_clip':[None, 1.02]
    }
}
params = middle_trend_params[1]
df_trend, middle_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(middle_trend_dict))
middle_trend_dict

37


{10001: 1.0129351534528772,
 1095: 1.008447263901533,
 1125: 1.0154674931169532,
 13133: 1.0148110803992443,
 13285: 1.02,
 16021: 1.02,
 16051: 1.0102600596450686,
 17049: 1.0145852062314247,
 17119: 1.0160684278423038,
 17135: 1.02,
 18073: 1.02,
 21167: 1.0087742823839232,
 21205: 1.0089322199917075,
 23017: 1.02,
 24023: 1.02,
 27161: 1.0095260836940123,
 29209: 0.9900118427149365,
 30047: 1.0077290823975364,
 30063: 1.007380223819338,
 32017: 0.989911033357664,
 33009: 1.0036445676092536,
 37071: 1.0116879058571973,
 37097: 1.008211996600611,
 37161: 1.0085466949373592,
 37175: 1.0167317606498527,
 37195: 1.0099083923055931,
 39101: 1.014221543488473,
 42013: 0.984723291353719,
 46083: 1.008549269096111,
 48051: 1.0054762585979062,
 49057: 1.0060150299560793,
 51033: 1.0088285129605443,
 51103: 1.0073430500039617,
 53007: 1.02,
 53065: 0.9778049488905132,
 6005: 0.9940137900855888,
 8037: 1.0155565410397998}

In [164]:
lowmiddle_trend_params = {
    1: {
        'n':12,
        'thre':12,
        'thre_r':0,
        'lower_bound': 140,
        'upper_bound': 1000,
        'use_regularize': True,
        'v_regularize': [0.001, 0],
        'v_clip':[None, 1.03]
    }
}
params = lowmiddle_trend_params[1]
df_trend, lowmiddle_trend_dict = preprocess.get_trend_dict(df_all, **params)
print(len(lowmiddle_trend_dict))
lowmiddle_trend_dict

3


{17135: 1.03, 26085: 1.0274230177900308, 28051: 1.03}

In [165]:
for keys in lowmiddle_trend_dict.keys():
    if keys in middle_trend_dict.keys():
        print(keys)

17135


In [166]:
df_sub1 = pd.read_csv('../submission/submission_13820.csv', index_col='row_id')
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, trend_dict)
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, high_trend_dict, method='replace')
df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, middle_trend_dict, method='replace')
# df_sub1, df_extract, var_dict = utils.insert_trend(df_sub1, df_all, df_census, lowmiddle_trend_dict, method='replace')
df_sub1.to_csv('../submission/submission_13820_trend.csv')

In [167]:
df_merged = utils.compare_submission(df_sub1, 'submission_13820')
df_diff = df_merged[df_merged['smape']>0.001].copy()
df_diff.shape

(70, 6)

In [168]:
df_merged['smape'].sum()

45.78760747865825

In [125]:
47
25.251103

25.251103

In [126]:
df_merged[df_merged['microbusiness_density']<df_merged['baseline']]

,row_id,microbusiness_density,cfips,month,baseline,smape
186,6005_2023-01-01,13.309681,6005,2023-01-01,13.398735,0.666858
480,13193_2023-01-01,0.921605,13193,2023-01-01,0.922528,0.100050
707,18025_2023-01-01,1.314474,18025,2023-01-01,1.315789,0.100050
1584,29209_2023-01-01,5.464115,29209,2023-01-01,5.525793,1.122451
1752,32017_2023-01-01,73.652821,32017,2023-01-01,74.396026,1.004000
2247,42013_2023-01-01,3.027983,42013,2023-01-01,3.071815,1.437174
2978,53065_2023-01-01,4.503120,53065,2023-01-01,4.599392,2.115277
2996,54023_2023-01-01,0.763968,54023,2023-01-01,0.764732,0.100050


In [14]:
# df_submission = pd.read_csv('../submission/submission_2023-02-27_16_35_41.csv', index_col='row_id')
df_submission = pd.read_csv('../submission/submission_2023-02-28_18_21_26.csv', index_col='row_id')
# df_submission = pd.read_csv('../submission/submission_2023-02-27_19_41_33.csv', index_col='row_id')
# df_submission = pd.read_csv('../submission/submission_2023-02-27_21_15_16.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_13827')
print(df_merged['smape'].mean())

0.016963210799712435
